In [1]:
import pygame
from pygame import*
import numpy as np
from math import pi
import random
from world import *
import os
from time import sleep
from collections import deque
from keras.layers.core import Dense,Activation,Flatten,Dropout
from keras.layers import Input
from keras.optimizers import Adam
from keras.models import Model

np.random.seed(23)
random.seed(23)
m = generate_map(8, 6, 3, 3)          #создаем карту

Using TensorFlow backend.


In [2]:
class MyGame(object):
    def __init__(self):
        #os.environ["SDL_VIDEODRIVER"] = "dummy"
        pygame.init()
        self.sizeshow = (800,600)                                              #размер экрана
        self.background = pygame.Surface(self.sizeshow)                        # фон
        self.window = pygame.display.set_mode(self.sizeshow)                   #создаем экран
        self.len_step=0.1                                                      #размер шага
        self.start_phase=pi*3/4                                                #начальное направление
        self.map=m                                                             # наша карта
        self.start_pos=State((self.map[0][0] + self.map[0][1]) / 2,rect(self.len_step, self.start_phase))#положение
        self.car=create_car()                                                  #объект машинки
        self.map_midle=np.mean(np.array(self.map),axis=0)[0]                   #центр карты
        self.num_layers=7                                                      #колличество лучей
        self.white = (255,255,255)
    def reset(self):
        self.lit = []
        self.car_position=self.start_pos.copy()
        self.phase = 100
    def step(self,step):
        self.car_position = neuron(self.car_position,step)
        self.lit = ray(self.car,self.map, self.car_position, self.num_layers, True, self.background)
        self.car_position.position = self.car_position.position + self.car_position.direction
        self.phase,self.exity = carr(self.map_midle, self.car_position, self.phase)
        #начинаем рисовать
        plot_map(self.map, self.background)
        self.window.blit(self.background,(0,0))                                #натягиваем фон
        self.background.fill(self.white)                                       #заливаем фон белым цветом
        pygame.display.update()
        sleep(0.1)
        #закончили рисовать
        if is_out_of_map(self.map, self.car_position.position) or self.exity:       #если произошло столкновение с картой
            return np.array(self.lit), True, self.phase
        else:
            return np.array(self.lit), False, self.phase

In [8]:
def get_next_batch(experince,model,num_actions,gamma):
    batch_size = len(experince)
    X = np.zeros((batch_size,7))
    Y = np.zeros((batch_size,num_actions))
    step = 0
    for i in range(1,batch_size+1):
        s_t,a_t,s_tp1,game_over = experince[-i]
        X[-i] = s_t
        Y[-i] = model.predict(s_t.reshape(1,7))[0]            ###
        Q_sa = np.max(model.predict(s_tp1.reshape(1,7))[0])  ##
        if step < 7:
            Y[-i,a_t] = -1
            step += 1
        else:
            Y[-i,a_t] = 1.0 + abs(Q_sa)
    return X,Y

In [3]:
def get_model():    
    inp=Input(shape=(7,))
    x=Dense(512,kernel_initializer='normal')(inp)
    x=Activation('relu')(x)
    x=Dense(512,kernel_initializer='normal')(x)
    x=Activation('relu')(x)
    x=Dense(256,kernel_initializer='normal')(x)
    x=Activation('relu')(x)
    out=Dense(3,kernel_initializer='normal')(x)
    model=Model(inputs=inp,outputs=out,name='autodriver')
    model.compile(optimizer=Adam(lr=1e-6),loss='mse')
    return model

In [9]:
NUM_ACTIONS = 3
GAMMA = 0.99
INITIAL_EPSILON = 0.1
FINAL_EPSILON = 0.0001
MEMORY_SIZE = 7000
NUM_EPOCHS_OBSERVE = 0
NUM_EPOCHS_TRAIN = 15000

BATCH_SIZE = 32
NUM_EPOCHS = NUM_EPOCHS_OBSERVE + NUM_EPOCHS_TRAIN

game = MyGame()
experince = deque(maxlen=MEMORY_SIZE)
max_step = 1762
epsilon = INITIAL_EPSILON

In [ ]:
for e in range(NUM_EPOCHS):
    game.reset()
    loss = 0.0
    a_0=1
    x_t,game_over,alpha = game.step(a_0)
    s_t = x_t
    game_over = False
    num_steps = 0
    experince.clear()
    while not game_over:
        s_tml = s_t
        #следующее действие
        if e <= NUM_EPOCHS_OBSERVE:
            a_t = np.random.randint(low=0,high=NUM_ACTIONS,size=1)[0]
        else:
            q = model.predict(s_t.reshape(1,7))[0]                      ####
            a_t = np.argmax(q)
        # применить действие к игре
        x_t, game_over,alpha=game.step(a_t)
        s_t = x_t
        # увеличить num_steps
        num_steps += 1
        #сохраним опыт
        experince.append((s_tml, a_t, s_t, game_over))

    if num_steps > max_step:
        model.save_weights("game_model_weight_ray_7_{:04d}.HDF5".format(num_steps))
        print("SAVE_MODEL")
        max_step = num_steps

    X,Y = get_next_batch(experince,model,NUM_ACTIONS,GAMMA)
    for i in range(5):
        loss += model.train_on_batch(X,Y)
    print("Epoch {:04d}/{:d} | Loss {:.5f} | Steps Count {:04d}  |  Alpha - {:04f}".format(e + 1,NUM_EPOCHS,loss,num_steps,alpha))

Epoch 0001/15000 | Loss 14.31371 | Steps Count 0026  |  Alpha - 342.734191
SAVE_MODEL
Epoch 0002/15000 | Loss 2.97160 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0003/15000 | Loss 2.94650 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0004/15000 | Loss 2.92055 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0005/15000 | Loss 2.89363 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0006/15000 | Loss 2.86600 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0007/15000 | Loss 2.83807 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0008/15000 | Loss 2.81008 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0009/15000 | Loss 2.78238 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0010/15000 | Loss 2.75575 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0011/15000 | Loss 2.73100 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0012/15000 | Loss 2.70281 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0013/15000 | Loss 2.67788 | Steps Count 1762  |  Alpha - 207.141282
Epoch 0014/15000 | Loss 2.

Epoch 0112/15000 | Loss 1.85244 | Steps Count 0290  |  Alpha - 200.274380
Epoch 0113/15000 | Loss 1.85286 | Steps Count 0290  |  Alpha - 200.274380
Epoch 0114/15000 | Loss 1.85330 | Steps Count 0290  |  Alpha - 200.274380
Epoch 0115/15000 | Loss 1.85376 | Steps Count 0290  |  Alpha - 200.274380
Epoch 0116/15000 | Loss 1.85424 | Steps Count 0290  |  Alpha - 200.274380
Epoch 0117/15000 | Loss 1.85474 | Steps Count 0290  |  Alpha - 200.274380


In [ ]:
game = 
game.reset()
x_t, game_over,alpha = game.step(1)
step = 0
while not game_over:
    a_t = np.argmax(model.predict(x_t.reshape(1,7)))
    x_t,  game_over,alpha = game.step(a_t)
    step += 1
print("Step = {:04d}  |  Alpha = {:04f}".format(step,alpha))

In [4]:
model = get_model()
model.load_weights("game_model_weight_ray_7_1762.HDF5")